In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from catboost import CatBoostClassifier, Pool
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
import pymorphy2
from scipy.sparse import hstack

import warnings
warnings.filterwarnings("ignore")

In [2]:
Path("models").mkdir(exist_ok=True)
Path("results").mkdir(exist_ok=True)

### Exploratory Data Analysis

In [4]:
train = pd.read_csv("data/train_dataset_train.csv")

In [4]:
train.head()

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
0,2246,Помогите начальник Льговского рэс не реагирует...,"Нарушения, связанные с содержанием электросети...",Администрация Льговского района,3
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,Аварийные деревья,Администрация города Курска,3
2,2240,Агресивные собаки. На радуге там стая из подро...,Безнадзорные животные,Администрация города Курска,1
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,Нескошенная сорная растительность в местах общ...,Комитет дорожного хозяйства города Курска,3
4,1797,<p style=`text-align:justify;`><span style=`ba...,Аварийные деревья,Комитет городского хозяйства города Курска,3


In [5]:
train['Текст Сообщения'].values[0]

'Помогите начальник Льговского рэс не реагирует на жалобы, а мы как малейший ветер сидим без света, а именно в деревне большие угоны улица старая слобода. Пожалуйста помогите, пускай вычистит и фотоотчет сделает'

In [6]:
train['Текст Сообщения'].values[4]

'<p style=`text-align:justify;`><span style=`background-color:white;`>Здравствуйте! Рядом с&nbsp;домом 1 «А» по&nbsp;улице&nbsp;Светлая, вне придомовой территории, растет клен. Это довольно старое и хрупкое дерево. В&nbsp;случае непогоды возможно обламывание ветвей (как уже неоднократно случалось), что с большой вероятностью приведет к&nbsp;обрыву находящихся рядом электрических проводов, в&nbsp;случае повреждения которых без света останется вся улица. Кроме того, возникает угроза жизни и&nbsp;здоровью проживающих рядом людей (в том числе детей), а&nbsp;также их автомобилям и другому имуществу. Прошу провести спил дерева или его санитарную обрезку.</span></p>'

In [5]:
# Количество уникальных тематик
train['Тематика'].nunique()

161

In [6]:
# Количество ответственных лиц
train['Ответственное лицо'].nunique()

75

In [7]:
# Количество уникальных категорий
train['Категория'].nunique()

17

In [10]:
# Дубликаты
len(train[train[['Текст Сообщения', 'Тематика', 'Ответственное лицо', 'Категория']].duplicated()])

17

In [15]:
ids = train[train[['Текст Сообщения', 'Тематика', 'Ответственное лицо', 'Категория']].duplicated()].id.unique()
train = (train[
    ~train.id.isin(ids)
    ]
)

In [16]:
# Пропуски
train.isna().sum()

id                    0
Текст Сообщения       0
Тематика              0
Ответственное лицо    0
Категория             0
dtype: int64

In [17]:
train

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория
0,2246,Помогите начальник Льговского рэс не реагирует...,"Нарушения, связанные с содержанием электросети...",Администрация Льговского района,3
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,Аварийные деревья,Администрация города Курска,3
2,2240,Агресивные собаки. На радуге там стая из подро...,Безнадзорные животные,Администрация города Курска,1
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,Нескошенная сорная растительность в местах общ...,Комитет дорожного хозяйства города Курска,3
4,1797,<p style=`text-align:justify;`><span style=`ba...,Аварийные деревья,Комитет городского хозяйства города Курска,3
...,...,...,...,...,...
1995,1356,<p>22.12.21 в вечернее время появилась вонь от...,Неприятные запахи,Комитет природных ресурсов Курской области,16
1996,243,<p>Добрый день! Хочу сообщить о такой проблеме...,Парковки на дорогах в границах городских округ...,Администрация города Курска,0
1997,2350,Состоят 3 засохшие дерева (2 большие берёзы и...,Аварийные деревья,Администрация города Курска,3
1998,1937,"Пожалуйста роман Владимирович, в скором времен...",Нарушение дорожного покрытия (ямы) на дорогах...,Администрация города Курска,0


In [18]:
# Топ обращений по тематике
train.groupby("Тематика", as_index=False)['Категория'].count().sort_values("Категория", ascending=False).head(10)

,Тематика,Категория
27,Нарушение дорожного покрытия (ямы) на дорогах...,152
78,Неработающее наружное освещение,104
131,Отсутствие люков на коммуникационных колодцах,88
82,Несанкционированные свалки твёрдых бытовых отх...,81
2,Аварийные деревья,79
35,Нарушение теплоснабжения многоквартирного дома,58
65,Некачественное водоснабжение многоквартирного ...,51
96,Неудовлетворительная уборка улиц и тротуаров,49
52,Не соответствующий установленным нормам сбор и...,46
138,Очистка от снега и наледи дорог в границах гор...,44


In [19]:
# Количество обращений по категориям
# Есть сильный дисбаланс классов
train.groupby("Категория", as_index=False)['Тематика'].count().sort_values("Тематика", ascending=False).head(10)

,Категория,Тематика
3,3,944
0,0,475
16,16,145
8,8,139
4,4,108
10,10,48
7,7,27
1,1,25
11,11,19
5,5,12


In [20]:
pd.set_option('display.max_colwidth', 200)  

In [21]:
(train[train['Категория']==3]
.groupby("Тематика", as_index=False)['Текст Сообщения'].count()
.sort_values("Текст Сообщения", ascending=False).head(20))

,Тематика,Текст Сообщения
27,Неработающее наружное освещение,104
44,Отсутствие люков на коммуникационных колодцах,88
1,Аварийные деревья,79
5,Нарушение теплоснабжения многоквартирного дома,58
20,Некачественное водоснабжение многоквартирного дома,51
36,Неудовлетворительная уборка улиц и тротуаров,49
12,Не соответствующий установленным нормам сбор и вывоз твёрдых бытовых отходов в районах многоквартирных домов,46
3,Длительное неисполнение заявок управляющей компанией,34
39,Неудовлетворительное содержание детской (спортивной) площадки на территории многоквартирных домов,32
31,Несвоевременный (некачественный) текущий ремонт многоквартирного дома,30


In [22]:
(train[train['Категория']==0]
.groupby("Тематика", as_index=False)['Текст Сообщения'].count()
.sort_values("Текст Сообщения", ascending=False).head(20))

,Тематика,Текст Сообщения
7,Нарушение дорожного покрытия (ямы) на дорогах в границах городских округов и сельских поселений,152
17,Очистка от снега и наледи дорог в границах городских округов и сельских поселений,44
19,Парковки на дорогах в границах городских округов и сельских поселений,38
15,Отсутствие твёрдого дорожного покрытия на дорогах в границах городских округов и сельских поселений,38
23,Светофоры на дорогах в границах городских округов и сельских поселений,36
20,Пешеходные переходы на дорогах в границах городских округов и сельских поселений,36
2,Дорожные знаки на дорогах в границах городских округов и сельских поселений,25
11,Освещение дорог в границах городских округов и сельских поселений,24
0,Безопасная дорога в школу на дорогах в границах городских округов и сельских поселений,11
12,Освещение дорог регионального и межмуниципального значения,10


In [23]:
(train[train['Категория']==16]
.groupby("Тематика", as_index=False)['Текст Сообщения'].count()
.sort_values("Текст Сообщения", ascending=False).head(20))

,Тематика,Текст Сообщения
5,Несанкционированные свалки твёрдых бытовых отходов,81
2,Неприятные запахи,44
6,Несанкционированный сброс жидких бытовых отходов,5
7,Порча земель,5
0,Загрязнение водных объектов,4
1,Нарушение правил пожарной безопасности в сфере природопользования,2
3,Несанкционированная вырубка зелёных насаждений в лесопарковых зонах,2
4,Несанкционированная вырубка зелёных насаждений в населённых пунктах,2


In [24]:
(train[train['Категория']==8]
.groupby("Тематика", as_index=False)['Текст Сообщения'].count()
.sort_values("Текст Сообщения", ascending=False).head(20))

,Тематика,Текст Сообщения
8,Недостаточное количество транспорта на маршруте при осуществлении муниципальных (пригородных) перевозок,33
3,Нарушение правил посадки и высадки пассажиров при осуществлении муниципальных (пригородных) перевозок,32
14,Неудовлетворительное санитарное состояние транспорта при осуществлении муниципальных (пригородных) перевозок,21
15,Неудовлетворительное санитарное состояние транспорта при осуществлении внутриобластных междугородных перевозок,8
11,Неудобный график движения при осуществлении муниципальных (пригородных) перевозок,7
5,"Нарушения, связанные с оплатой проезда при осуществлении муниципальных (пригородных) перевозок",7
9,Необходимо назначение дополнительных поездов,5
13,Неудовлетворительное санитарное или техническое состояние вагонов,5
12,Неудобный график движения пригородного ж/д транспорта,5
2,Нарушение правил посадки и высадки пассажиров при осуществлении внутриобластных междугородных перевозок,4


In [25]:
train[['Текст Сообщения', 'Тематика']].sample(20)

,Текст Сообщения,Тематика
1381,"Вот так из рук вон плохо убирают мусор во дворах домов по улице Черняховского 20,20б,22",Не соответствующий установленным нормам сбор и вывоз твёрдых бытовых отходов в районах многоквартирных домов
1877,"г.Щигры ул.Лазарева, 7Д. В 5-7 метрах от дома - почти непроходимый лес. Тополя и другие деревья, возраст которых стремится к 100, опасны тем, что могут просто рухнуть на людей, а также - на высок...",Аварийные деревья
1390,Твёрдые бытовые отходы что делать людям у которых печное отопление какая разница где мусор будет гореть в городе на свалке или у меня дома так за это за все я еще должен заплатить а потом когда вс...,Не соответствующий установленным нормам сбор и вывоз твёрдых бытовых отходов с территории частного сектора
1132,<p>Вот такой чудо бурьян растет рядом с домом №2 на протяжени лета и ни кому до него нет дела.Этот травяной бурьян начинается от начало дома и тянется почти до ТЦ Европа.Надо наверное руководство ...,Нескошенная сорная растительность в местах общего пользования в районах частного сектра
1919,"<p>Здравствуйте!&nbsp;</p><p>Я несовершеннолетний, могу ли я пойти в Cental Park для покупки вещей. Я несовершеннолетний и вакцинироваться мне нельзя, а вещи покупать надо. Из близкого окружения е...",Я хочу задать вопрос о вакцинации
1678,"<p>Здравствуйте, хочу попросить разобраться запахом канализации около нашего дома, звонила в водоканал несколько раз, никак не отреагировали. Вонь идёт из многочисленных колодцев, рядом проходит к...",Неприятные запахи
505,"<p>Здравствуйте,сейчас ведутся работы по реконструкции моста,дорожные работники живут около частного дома.Подьезд к частному дому раскатали, дорогу всю разбили.Кто теперь должен ремонтировать доро...",Нарушение дорожного покрытия (ямы) на дорогах регионального и межмуниципального значения
887,"Прошу навести порядок. Урна полная мусора, рядом с урной мусор, на остановке мусор, за остановкой тоже мусор. Прошу навести порядок",Несанкционированные свалки твёрдых бытовых отходов
1738,"Добрый день. Комитет социальной защиты населения оказывает адресную материальную помощь нуждающимся семьям. Я получаю пособие на детей 174 рубля 50 коп., у меня трое детей. Мне стало известно что ...","Нарушения порядка предоставления мер социальной поддержки, в т.ч. адресной помощи"
1948,"<p>Ужасное состояние тротуарной плитки,с коляской невозможно проехать,да и обычным пешеходам тоже</p>",Нарушение дорожного покрытия (ямы) на дорогах в границах городских округов и сельских поселений


In [26]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nastyusha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
def clean_text(text):
    
    text = BeautifulSoup(str(text), "html.parser").get_text()
    text = text.replace('&nbsp;', ' ').replace('\xa0', ' ')
    text = re.sub(r'[^а-яА-ЯёЁ\s]', ' ', text)
    text = text.lower().strip()
    tokens = text.split()
    tokens = [word for word in tokens if word not in russian_stopwords]
    
    return ' '.join(tokens)

In [28]:
train['cleaned_text'] = train['Текст Сообщения'].apply(clean_text)
train[['Текст Сообщения', 'cleaned_text']].sample(20)

,Текст Сообщения,cleaned_text
1862,Прошу почистить ливневки на Майском бульваре,прошу почистить ливневки майском бульваре
702,"<p>Добрый день ! Каждый год (!!!), весной, и когда дожди идут на стоянке Бумеранга образуется &nbsp;огромная лужа! Машины плавают в ней. &nbsp;Из-за наледи, весной буксуют в луже и застревают. Жив...",добрый день каждый год весной дожди идут стоянке бумеранга образуется огромная лужа машины плавают наледи весной буксуют луже застревают живу доме напротив тц бумеранг вижу постоянно водители пото...
1265,"По центральной дороге на повороте есть крутой обрыв , не знав дорогу можно с него слететь. Стоит один указатель `поворот`, мало кто обращает внимания на знаки.можно же поставить хоть какой нибудь...",центральной дороге повороте крутой обрыв знав дорогу слететь стоит указатель поворот мало обращает внимания знаки поставить отбойник правильно называется
1550,"<p>Прошу управляющую компанию очистить от снега дворовую территорию по ул. Красной в домах 105,91,93, 99,101, 101А,109. О проделанной работе прошу выслать фото отчёт.&nbsp;</p>",прошу управляющую компанию очистить снега дворовую территорию ул красной домах проделанной работе прошу выслать фото отчёт
299,"<p>Здравствуйте, почему 1 ые классы не допускают до учёбы в обычном режиме, чему может ребёнок в 1 - м классе научиться дистанционно!!! А если родители работают с кем должен быть ребёнок дома???!!...",здравствуйте почему ые классы допускают учёбы обычном режиме чему ребёнок м классе научиться дистанционно родители работают кем должен ребёнок дома нужно писать распоряжения законы просто примите ...
523,В квитанциях за июль месяц тариф на водоотведение холодной воды на 9% выше водоотведения горячей. При этом эта же услуга по ОДН тарифицируется для горячей и холодной воды одинаково. Каким образом ...,квитанциях июль месяц тариф водоотведение холодной воды выше водоотведения горячей эта услуга одн тарифицируется горячей холодной воды одинаково каким образом отводить воду разным тарифам вся утил...
690,<p>не работает наружное освещение &nbsp;до дома № 24</p>,работает наружное освещение дома
98,"<p>недавно получил ключи от квартиры в новом доме. 1й подъезд в этом доме еще не сдался. в каждой квартире на батареях стоят приборы учета тепла. как только я получил ключи, батареи у меня работаю...",недавно получил ключи квартиры новом доме й подъезд доме сдался каждой квартире батареях стоят приборы учета тепла получил ключи батареи работают полной мощности прикрыл вентиля батареях живу ук п...
110,"<p>Разбит канализационный люк. Проезжая часть, она же пешеходная. Требуется замена.</p>",разбит канализационный люк проезжая часть пешеходная требуется замена
1218,"Постоянно проезжающие `служебные машины` уничтожают асфальтовое покрытие в парке Боевка. Невозможно совершать пешие прогулки. Нужно запретить въезд на территорию любого транспорта, кроме спец маши...",постоянно проезжающие служебные машины уничтожают асфальтовое покрытие парке боевка невозможно совершать пешие прогулки нужно запретить въезд территорию любого транспорта кроме спец машин служб


In [29]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize_text(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

In [30]:
train['lemmatized_text'] = train['cleaned_text'].apply(lemmatize_text)

In [31]:
train[['Текст Сообщения', 'lemmatized_text']]

,Текст Сообщения,lemmatized_text
0,"Помогите начальник Льговского рэс не реагирует на жалобы, а мы как малейший ветер сидим без света, а именно в деревне большие угоны улица старая слобода. Пожалуйста помогите, пускай вычистит и фот...",помочь начальник льговский рэс реагировать жалоба малый ветер сидеть свет именно деревня больший угон улица старый слобода пожалуйста помочь пускай вычистить фотоотчёт сделать
1,"<p>По фасаду дома по адресу ул. Урицкого 22 проходит труба газовой магистрали. Также эта труба проходит рядом с деревом, которое при каждом порыве ветра цепляет трубку, заставляя ее двигаться по к...",фасад дом адрес ул урицкий проходить труба газовый магистраль также этот труба проходить рядом дерево который каждый порыв ветер цеплять трубка заставлять двигаться крепление бояться особенно силь...
2,Агресивные собаки. На радуге там стая из подросших щенков и зврослых собак. Они на Ребенка бросилась. Соьаки бросаются пытаются за ноги укусить. Разговор с охранниками не дал результатов,агресивный собака радуга стая подрасти щенок зврослый собака ребёнок броситься соьаки бросаться пытаться нога укусить разговор охранник дать результат
3,"<p>На пересечении &nbsp;улиц Сосновская и Береговая &nbsp;завалено все песком и гравием на санитарной зоне участка который относится к <span style=`background-color:rgb(255,255,255);color:rgb(51,5...",пересечение улица сосновский береговой завалить песок гравий санитарный зона участок который относиться муп кгтпо центральный рынок невозможно пройти проехать коляска
4,"<p style=`text-align:justify;`><span style=`background-color:white;`>Здравствуйте! Рядом с&nbsp;домом 1 «А» по&nbsp;улице&nbsp;Светлая, вне придомовой территории, растет клен. Это довольно старое ...",здравствуйте рядом дом улица светлый вне придомовый территория расти клён это довольно старое хрупкий дерево случай непогода возможно обламывание ветвь неоднократно случаться большой вероятность п...
...,...,...
1995,<p>22.12.21 в вечернее время появилась вонь от Грибной радуги</p>,вечерний время появиться вонь грибной радуга
1996,"<p>Добрый день! Хочу сообщить о такой проблеме - возле нашего дома по адресу г.Курск, ул. Косухина 24 в торце дома и перед 1м подъездом находятся незаконные гаражи. &nbsp;У нас очень хорошая управ...",добрый день хотеть сообщить проблема возле наш дом адрес г курск ул косухин торец дом м подъезд находиться незаконный гараж очень хороший управлять компания против построить дополнительный парково...
1997,Состоят 3 засохшие дерева (2 большие берёзы и еще одно дерево) возле детской площадки между домами 3 и 5 на проспекте Энтузиастов,состоять засохнуть дерево больший берёза один дерево возле детский площадка дом проспект энтузиаст
1998,"Пожалуйста роман Владимирович, в скором времени тут на а.д. Курск-п.Искра уже многие остались без колёс, в день по 10-15 машин остаются без колёс. Просимммммммм скорее решить вопрос.",пожалуйста роман владимирович скорый время далее курск п искра многие остаться колесо день машина оставаться колесо просимммммммм скорее решить вопрос


### Logistic Regression + TF-IDF

In [38]:
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    lowercase=False  
)

In [39]:
X_tfidf = vectorizer.fit_transform(train['lemmatized_text'])

In [40]:
print(f"Размер матрицы TF-IDF: {X_tfidf.shape}")
print(f"Количество уникальных слов в словаре: {len(vectorizer.vocabulary_)}")

Размер матрицы TF-IDF: (1983, 10000)
Количество уникальных слов в словаре: 10000


In [41]:
cat_features = train[['Тематика', 'Ответственное лицо']]

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_cat = encoder.fit_transform(cat_features)

In [42]:
X_combined = hstack([X_tfidf, X_cat])

In [43]:
le_category = LabelEncoder()
train['label'] = le_category.fit_transform(train['Категория'])

In [44]:
y = train[['label']]  

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y,
    test_size=0.2,
    random_state=42
)

In [45]:
model = LogisticRegression(
    max_iter=1000, random_state=42,
    class_weight='balanced',
    solver='lbfgs',
    penalty='l2',
    C=1.0,
    multi_class='multinomial'
)
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000,
                   multi_class='multinomial', random_state=42)

In [46]:
y_pred = model.predict(X_test)


acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {acc:.4f}")
print(f"Macro-F1: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9673
Macro-F1: 0.6905

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99        82
           1       1.00      1.00      1.00         4
           2       0.00      0.00      0.00         1
           3       0.96      0.99      0.98       203
           4       0.96      0.93      0.95        28
           5       1.00      1.00      1.00         1
           6       1.00      0.50      0.67         2
           7       1.00      1.00      1.00         7
           8       0.91      0.95      0.93        22
           9       0.67      1.00      0.80         2
          10       1.00      0.92      0.96        12
          11       1.00      0.67      0.80         3
          12       0.00      0.00      0.00         1
          13       0.50      1.00      0.67         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       1.0

In [47]:
joblib.dump(model, "models/logreg_model.pkl")
joblib.dump(vectorizer, "models/tfidf_vectorizer.pkl")
joblib.dump(encoder, "models/cat_encoder.pkl")

['models/cat_encoder.pkl']

In [48]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - Logistic Regression")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("results/cm_logreg.png")
plt.close()

In [49]:
results_file = "results/comparison.csv"
if not os.path.exists(results_file):
    results_df = pd.DataFrame(columns=["Model", "Accuracy", "Macro-F1"])
else:
    results_df = pd.read_csv(results_file)

new_row = pd.DataFrame([{
    "Model": "Logistic Regression + TF-IDF + CatFeatures",
    "Accuracy": acc,
    "Macro-F1": f1
}])

results_df = pd.concat([results_df, new_row], ignore_index=True)
results_df.to_csv(results_file, index=False)

## Catboost

In [50]:
y = train['Категория']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    train[['cleaned_text', 'Тематика', 'Ответственное лицо']],
    y,
    test_size=0.2,
    random_state=42,
)

In [52]:
text_features = ['cleaned_text']
cat_features = ['Тематика', 'Ответственное лицо']

train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=text_features,
    cat_features=cat_features
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    text_features=text_features,
    cat_features=cat_features
)


In [53]:
param_grid = {
    'iterations': [100, 200],
    'depth': [4, 6],
    'learning_rate': [0.01, 0.1],
    'l2_leaf_reg': [1, 3]
}

In [54]:
best_model = CatBoostClassifier(
    verbose=0,
    task_type="CPU",
    loss_function='MultiClass',
    eval_metric='TotalF1:average=Macro'
)

best_model.grid_search(
    param_grid,
    train_pool,
    cv=3,
    refit=True
)


bestTest = 0.3999541782
bestIteration = 97

0:	loss: 0.3999542	best: 0.3999542 (0)	total: 1.61s	remaining: 24.2s

bestTest = 0.624246448
bestIteration = 77

1:	loss: 0.6242464	best: 0.6242464 (1)	total: 3.5s	remaining: 24.5s

bestTest = 0.3366319195
bestIteration = 85

2:	loss: 0.3366319	best: 0.6242464 (1)	total: 4.95s	remaining: 21.5s

bestTest = 0.6161877646
bestIteration = 99

3:	loss: 0.6161878	best: 0.6242464 (1)	total: 6.41s	remaining: 19.2s

bestTest = 0.4070946994
bestIteration = 77

4:	loss: 0.4070947	best: 0.6242464 (1)	total: 9.47s	remaining: 20.8s

bestTest = 0.6140781784
bestIteration = 52

5:	loss: 0.6140782	best: 0.6242464 (1)	total: 12.3s	remaining: 20.6s

bestTest = 0.4070946994
bestIteration = 116

6:	loss: 0.4070947	best: 0.6242464 (1)	total: 15s	remaining: 19.3s

bestTest = 0.6432822457
bestIteration = 69

7:	loss: 0.6432822	best: 0.6432822 (7)	total: 17.7s	remaining: 17.7s

bestTest = 0.4070946994
bestIteration = 30

8:	loss: 0.4070947	best: 0.6432822 (7)	total: 


bestTest = 0.4394934979
bestIteration = 188

Training on fold [1/3]



bestTest = 0.4077520746
bestIteration = 24

Training on fold [2/3]



bestTest = 0.4381977785
bestIteration = 187



{'params': {'depth': 4,
  'learning_rate': 0.1,
  'l2_leaf_reg': 3,
  'iterations': 200},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [55]:
print("Лучшие параметры:", best_model.get_params())

Лучшие параметры: {'loss_function': 'MultiClass', 'verbose': 0, 'eval_metric': 'TotalF1:average=Macro', 'task_type': 'CPU', 'depth': 4, 'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 200}


In [56]:
y_pred = best_model.predict(test_pool)

In [57]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {acc:.4f}")
print(f"Macro-F1: {f1:.4f}")

Accuracy: 0.9521
Macro-F1: 0.5064


In [58]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        82
           1       1.00      1.00      1.00         4
           2       0.00      0.00      0.00         1
           3       0.97      1.00      0.99       203
           4       0.96      0.96      0.96        28
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         2
           7       1.00      0.14      0.25         7
           8       0.68      0.95      0.79        22
           9       0.00      0.00      0.00         2
          10       1.00      0.92      0.96        12
          11       0.67      0.67      0.67         3
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       1.00      1.00      1.00        26

    accuracy              

In [59]:
results_file = "results/comparison.csv"

if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
else:
    results_df = pd.DataFrame(columns=["Model", "Accuracy", "Macro-F1"])

new_row = pd.DataFrame([{
    "Model": "CatBoost + Text + CatFeatures",
    "Accuracy": acc,
    "Macro-F1": f1
}])

results_df = pd.concat([results_df, new_row], ignore_index=True)
results_df.to_csv(results_file, index=False)

In [60]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - CatBoost")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("results/cm_catboost.png")

plt.close()

In [62]:
results_df

,Model,Accuracy,Macro-F1
0,Logistic Regression + TF-IDF + CatFeatures,0.967254,0.690484
1,CatBoost + Text + CatFeatures,0.952141,0.506430
